# Temporal Word Embeddings for Early Detection of Psychological Disorders on Social Media

## How to early detect psychological disorders on social media using temporal word embeddings

#### Abastract
*Mental health disorders represent a public health challenge, where early detection is critical to mitigating adverse outcomes for individuals and society. The study of language and behavior is a pivotal component in mental health research, and the content from social media platforms serves as a valuable tool for identifying signs of mental health risks. This paper presents a novel framework leveraging temporal word embeddings to capture linguistic changes over time. We specifically aim at at identifying emerging psychological concerns on social media. By adapting temporal word representations, our approach quantifies shifts in language use that may signal mental health risks. To that end, we implement two alternative temporal word embedding models to detect linguistic variations and exploit these variations to train early detection classifiers. Our experiments, conducted on 18 datasets from the eRisk initiative (covering signs of conditions such as depression, anorexia, and self-harm), show that simple models focusing exclusively on temporal word usage patterns achieve competitive performance compared to state-of-the-art systems. Additionally, we perform a word-level analysis to understand the evolution of key terms among positive and control users. These findings underscore the potential of time-sensitive word models in this domain, being a promising avenue for future research in mental health surveillance.*

## Models

#### **TWEC**

First we difine our temporal word embeddings models. The first model is `TWEC` (Temporal Word Embeddings with A Compass), which extends the Word2Vec model by incorporating temporal information. TWEC captures linguistic changes over time by leveraging the surrounding words

In [1]:
from gensim.models import callbacks


class MyCallback(callbacks.CallbackAny2Vec):
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print("Pérdida después de la época {}: {}".format(self.epoch, loss))
        else:
            print(
                "Pérdida después de la época {}: {}".format(
                    self.epoch, loss - self.loss_previous_step
                )
            )
        self.epoch += 1
        self.loss_previous_step = loss

In [ ]:
from gensim.models.word2vec import Word2Vec, LineSentence, PathLineSentences

from gensim import utils
import os
import numpy as np
import logging
import copy
from gensim.utils import tokenize
import multiprocessing
from tqdm import tqdm


class TWEC:
    """
    Handles alignment between multiple slices of temporal text
    """

    def __init__(
        self,
        size=100,
        sg=0,
        siter=10,
        ns=10,
        window=5,
        alpha=0.025,
        min_count=5,
        workers=2,
        test="test",
        init_mode="hidden",
    ):
        """

        :param size: Number of dimensions. Default is 100.
        :param sg: Neural architecture of Word2vec. Default is CBOW (). If 1, Skip-gram is employed.
        :param siter: Number of static iterations (epochs). Default is 5.
        :param diter: Number of dynamic iterations (epochs). Default is 5.
        :param ns: Number of negative sampling examples. Default is 10, min is 1.
        :param window: Size of the context window (left and right). Default is 5 (5 left + 5 right).
        :param alpha: Initial learning rate. Default is 0.025.
        :param min_count: Min frequency for words over the entire corpus. Default is 5.
        :param workers: Number of worker threads. Default is 2.
        :param test: Folder name of the diachronic corpus files for testing.
        :param init_mode: If \"hidden\" (default), initialize temporal models with hidden embeddings of the context;'
                            'if \"both\", initilize also the word embeddings;'
                            'if \"copy\", temporal models are initiliazed as a copy of the context model
                            (same vocabulary)
        """
        self.size = size
        self.sg = sg
        self.trained_slices = dict()
        self.gvocab = []
        self.epoch = siter
        self.negative = ns
        self.window = window
        self.static_alpha = alpha
        self.dynamic_alpha = alpha
        self.min_count = min_count
        self.workers = multiprocessing.cpu_count() - 3
        self.test = test
        self.init_mode = init_mode
        self.compass: None | Word2Vec = None

    def initialize_from_compass(self, model) -> Word2Vec:
        if self.compass is None:
            raise Exception("Compass model is not initialized")

        if self.init_mode == "copy":
            model = copy.deepcopy(self.compass)
        else:
            if self.compass.layer1_size != self.size:  # type: ignore
                raise Exception("Compass and Slice have different vector sizes")

            if len(model.wv.index_to_key) == 0:
                model.build_vocab(corpus_iterable=self.compass.wv.index_to_key)  # type: ignore

            vocab_m = model.wv.index_to_key

            indices = [
                self.compass.wv.key_to_index[w]
                for w in vocab_m
                if w in self.compass.wv.key_to_index
            ]
            new_syn1neg = np.array([self.compass.syn1neg[index] for index in indices])
            model.syn1neg = new_syn1neg

            if self.init_mode == "both":
                new_syn0 = np.array([self.compass.wv.syn0[index] for index in indices])  # type: ignore
                model.wv.syn0 = new_syn0

        model.learn_hidden = False  # type: ignore
        model.alpha = self.dynamic_alpha
        return model

    def internal_trimming_rule(self, word, count, min_count):
        """
        Internal rule used to trim words
        :param word:
        :return:
        """
        if word in self.gvocab:
            return utils.RULE_KEEP
        else:
            return utils.RULE_DISCARD

    def train_model(self, sentences) -> Word2Vec | None:
        model = None
        if self.compass is None or self.init_mode != "copy":
            model = Word2Vec(
                sg=self.sg,
                vector_size=self.size,
                alpha=self.static_alpha,
                negative=self.negative,
                window=self.window,
                min_count=self.min_count,
                workers=self.workers,
            )
            model.build_vocab(
                corpus_iterable=sentences,
                trim_rule=self.internal_trimming_rule
                if self.compass is not None
                else None,
            )

        if self.compass is not None:
            model = self.initialize_from_compass(model)
            model.train(
                corpus_iterable=sentences,
                total_words=sum([len(s) for s in sentences]),
                epochs=self.epoch,
                compute_loss=True,
            )
        else:
            model.train(
                corpus_iterable=sentences,
                total_words=sum([len(s) for s in sentences]),
                epochs=self.epoch,
                compute_loss=True,
                callbacks=[MyCallback()],
            )

        return model

    def train_compass(self, chunks):
        # sentences = [list(tokenize(frase)) for sentences in chunks for frase in sentences]
        sentences = [
            list(tokenize(text, lowercase=True, deacc=True)) for text in chunks
        ]
        print("Training the compass.")
        self.compass = self.train_model(sentences)
        print(f"Vocav -> {len(self.compass.wv.index_to_key)}")
        self.gvocab = self.compass.wv.index_to_key

    def train_slice(self, chunks):
        if self.compass is None:
            return Exception("Missing Compass")

        # sentences = [list(tokenize(frase)) for sentences in chunks for frase in sentences]
        sentences = [list(tokenize(frase)) for frase in chunks]
        model = self.train_model(sentences)
        return model

    # FINE TUNNING VARIATION

    def finetune_model(self, sentences, pretrained_path):
        model = None
        if self.compass is None or self.init_mode != "copy":
            model = Word2Vec(
                sg=self.sg,
                vector_size=self.size,
                alpha=self.static_alpha,
                negative=self.negative,
                window=self.window,
                min_count=self.min_count,
                workers=self.workers,
            )
            model.build_vocab(
                sentences,
                trim_rule=self.internal_trimming_rule
                if self.compass is not None
                else None,
            )
            # model.build_vocab(list(pretrained_model.vocab.keys()), update=True)
            model.intersect_word2vec_format(pretrained_path, binary=True, lockf=1.0)

        if self.compass is not None:
            model = self.initialize_from_compass(model)

        model.train(
            sentences,
            total_words=sum([len(s) for s in sentences]),
            epochs=self.epoch,
            compute_loss=True,
        )

        return model

    def finetune_compass(self, compass_text, pre_path, overwrite=False, save=True):
        sentences = PathLineSentences(compass_text)
        sentences.input_files = [
            s for s in sentences.input_files if not os.path.basename(s).startswith(".")
        ]
        logging.info("Finetunning the compass.")
        self.compass = self.finetune_model(sentences, pre_path)

        self.gvocab = self.compass.wv.index_to_key

    def finetune_slice(self, slice_text, pretrained):
        try:
            if self.compass is None:
                logging.info("Fuck where is the dam compass")
                return Exception("Missing Compass")
            logging.info(
                "Finetunning temporal embeddings: slice {}.".format(slice_text)
            )

            sentences = LineSentence(slice_text)
            model = self.finetune_model(sentences, pretrained)
            return model
        except Exception as fk:
            logging.error("What da > {}".format(fk))

#### **DCWE**

Then we define the `DCWE` model. This model combines BERT imput embeddings with temporal features the it pass this to BERT model for contextualization.

In [3]:
import torch
from torch import nn
from transformers.models.bert import BertForMaskedLM
from torch.nn import functional as F
from scipy.sparse import dok_matrix


def cosine_similarity(V1, V2):
    dot_prod = torch.einsum(
        "abc, cba -> ab", [V1, V2.permute(*torch.arange(V2.ndim - 1, -1, -1))]
    )
    norm_1 = torch.norm(V1, dim=-1)
    norm_2 = torch.norm(V2, dim=-1)
    return dot_prod / torch.einsum(
        "bc, bc -> bc", norm_1, norm_2
    )  # Scores de similitud entre embeddings estáticos y contextualizados


def isin(ar1, ar2):
    return (ar1[..., None] == ar2).any(-1)


class DCWE(nn.Module):
    def __init__(
        self,
        lambda_a,
        lambda_w,
        vocab_filter=torch.tensor([]),
        n_times=10,
        *args,
        **kwargs,
    ):
        super(DCWE, self).__init__()
        self.bert = BertForMaskedLM.from_pretrained("bert-base-uncased")
        self.bert_emb_layer = self.bert.get_input_embeddings()
        print(f"Model offset_components = {n_times}")
        self.offset_components = nn.ModuleList(
            [OffsetComponent() for _ in range(n_times)]
        )
        self.lambda_a = lambda_a
        self.lambda_w = lambda_w
        self.vocab_filter = vocab_filter

    # mlm_label, reviews, masks, segs, times, vocab_filter, SA
    def forward(self, reviews, times, masks, segs):
        bert_embs = self.bert_emb_layer(reviews)

        offset_last = torch.cat(
            [
                self.offset_components[int(j.item())](bert_embs[i])
                for i, j in enumerate(F.relu(times.detach().cpu() - 1))
            ],
            dim=0,
        )
        offset_now = torch.cat(
            [
                self.offset_components[int(j.item())](bert_embs[i])
                for i, j in enumerate(times.detach().cpu())
            ],
            dim=0,
        )
        offset_last = offset_last * (
            isin(reviews, self.vocab_filter)
        ).float().unsqueeze(-1).expand(-1, -1, 768)
        offset_now = offset_now * (isin(reviews, self.vocab_filter)).float().unsqueeze(
            -1
        ).expand(-1, -1, 768)

        input_embs = bert_embs + offset_now

        output = self.bert(
            inputs_embeds=input_embs,
            attention_mask=masks,
            token_type_ids=segs,
            output_hidden_states=True,
        )

        return offset_last, offset_now, output

    def loss(self, out, labels, function):
        offset_last, offset_now, output = out

        logits = output.logits
        loss = function(logits.view(-1, self.bert.config.vocab_size), labels.view(-1))
        loss += self.lambda_a * torch.norm(offset_now, dim=-1).pow(2).mean()
        loss += (
            self.lambda_w * torch.norm(offset_now - offset_last, dim=-1).pow(2).mean()
        )
        return loss

    def generate_deltas(self, texts, input_embs, output_embs, vocab_hash_map, deltas_f):
        sim_matrix = deltas_f(input_embs, output_embs).detach().cpu().numpy()

        chunk_mat = dok_matrix(
            (sim_matrix.shape[0], len(vocab_hash_map)), dtype=np.float32
        )
        aux_t = texts.cpu().numpy()

        for post_idx, post in enumerate(aux_t):
            for token in set(post):
                if token in vocab_hash_map:
                    indices = np.where(post == token)
                    # TODO estaba sum ahora mean
                    chunk_mat[post_idx, vocab_hash_map[token]] = np.mean(
                        sim_matrix[post_idx][indices]
                    )

        return chunk_mat


class OffsetComponent(nn.Module):
    def __init__(self):
        super(OffsetComponent, self).__init__()
        self.linear_1 = nn.Linear(768, 768)
        self.linear_2 = nn.Linear(768, 768)
        self.dropout = nn.Dropout(0.2)

    def forward(self, embs):
        h = self.dropout(torch.tanh(self.linear_1(embs)))
        offset = self.linear_2(h).unsqueeze(0)
        return offset


class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.linear_1 = nn.Linear(768, 100)
        self.linear_2 = nn.Linear(100, 1)
        self.dropout = nn.Dropout(0.2)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, sims):
        proj_1 = self.dropout(self.linear_1(sims))
        return torch.sigmoid(self.linear_2(proj_1))

### Deltas

Deltas is a measure of movement of word meaning in a text corpus. It's calculated with different similarity measures between the temporal version of the embeddings and the static version of the embeddings.

In [4]:
import torch


def jensen_shannon_divergence(p, q):
    m = 0.5 * (p + q)
    return 0.5 * (
        F.kl_div(m.log(), p, reduction="none").sum(-1)
        + F.kl_div(m.log(), q, reduction="none").sum(-1)
    )


def wasserstein_distance(u_values, v_values):
    def compute_wasserstein(u, v):
        u_sorter = torch.argsort(u, dim=-1)
        v_sorter = torch.argsort(v, dim=-1)

        all_values = torch.cat([u, v], dim=-1)
        all_values, _ = torch.sort(all_values, dim=-1)

        deltas = torch.diff(all_values, dim=-1)

        u_cdf = torch.searchsorted(
            torch.gather(u, -1, u_sorter), all_values[..., :-1], right=True
        )
        v_cdf = torch.searchsorted(
            torch.gather(v, -1, v_sorter), all_values[..., :-1], right=True
        )

        return torch.sum(torch.abs(u_cdf - v_cdf) * deltas, dim=-1)

    distances = torch.stack(
        [
            compute_wasserstein(u_values[:, i, :], v_values[:, i, :])
            for i in range(u_values.shape[1])
        ],
        dim=1,
    )

    return distances


def pairwise_distance(input_embs, output_embs, p=2):
    diff = input_embs - output_embs
    distances = torch.norm(diff, p=p, dim=-1)

    return distances


DISTANCES = {
    "cosine": lambda input_embs, output_embs: (
        1 - F.cosine_similarity(input_embs, output_embs, dim=-1)
    )
    / 2,
    "euclidean": lambda input_embs, output_embs: pairwise_distance(
        input_embs, output_embs, p=2
    ),
    "manhattan": lambda input_embs, output_embs: pairwise_distance(
        input_embs, output_embs, p=1
    ),
    "jensen_shannon": lambda input_embs, output_embs: jensen_shannon_divergence(
        input_embs, output_embs
    ),
    "wasserstein": lambda input_embs, output_embs: wasserstein_distance(
        input_embs, output_embs
    ),
    "chebyshev": lambda input_embs, output_embs: torch.max(
        torch.abs(input_embs - output_embs), dim=-1
    ).values,
    "minkowski": lambda input_embs, output_embs: pairwise_distance(
        input_embs, output_embs, p=3
    ).pow(1 / 3),
}


def f_generate_deltas(input_embs, output_embs, distance_metric="cosine"):
    if distance_metric in DISTANCES:
        sim_matrix = DISTANCES[distance_metric](input_embs, output_embs)
    else:
        raise ValueError(f"Unsupported distance metric: {distance_metric}")

    return sim_matrix

## Filters

Now that we have defined our models, we need to encapsulated then inside a class that implements BaseFilter interface and binds to the container.

In [ ]:
!pip install framework3


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [6]:
from framework3.base.base_clases import BaseFilter
from framework3.base.base_types import XYData
from framework3.container import Container

import pandas as pd


@Container.bind()
class TWECFilter(BaseFilter):
    def __init__(self, context_size: int):
        self.dCWE = TWEC(size=300, window=context_size)

    def fit(self, x: XYData, y: XYData | None) -> float | None:
        data: pd.DataFrame = x.value
        self.model.train_compass(data.text.values.tolist())
        self.vocab_hash_map = dict(
            zip(
                self.model.compass.wv.index_to_key,
                range(len(self.model.compass.wv.index_to_key)),
            )
        )

    def predict(self, x: XYData) -> XYData:
        data: pd.DataFrame = x.value

        all_deltas = {}
        for i, row in tqdm(enumerate(data.itertuples()), total=len(data.index)):
            tc = self.model.train_slice(row.text)
            for word in tc.wv.index_to_key:
                if word in self.vocab_hash_map:
                    j = self.vocab_hash_map[word]
                    # delta_matrix[int(i), j]

                    for metric in self.deltas_f:
                        aux = all_deltas.get(
                            metric,
                            dok_matrix(
                                (len(data.index), len(self.vocab_hash_map.items())),
                                dtype=np.float16,
                            ),
                        )
                        aux[int(i), j] = (
                            DISTANCES[metric](
                                torch.tensor([[self.model.compass.wv[word]]]),
                                torch.tensor([[tc.wv[word]]]),
                            )
                            .detach()
                            .cpu()
                            .numpy()
                        )
                        all_deltas[metric] = aux

        return XYData.mock(all_deltas)

ModuleNotFoundError: No module named 'tests.unit'

#### DCWE Needs data manipulation classes

In [ ]:
from collections import Counter

import torch
from torch.utils.data import Dataset
from nltk.corpus import stopwords

stops = set(stopwords.words("english"))


class TimeDatasetNoWindow(Dataset):
    def __init__(self, df: pd.DataFrame, tk, vocab_start=0, vocab_end=10000, **kwargs):
        self.vocab_start = vocab_start
        self.vocab_end = vocab_end
        df.dropna(inplace=True)
        df.reset_index(inplace=True, drop=True)

        self.tok = tk
        self.users = df.user.values
        self.deltas = df.chunk.values
        self.dates = df.date.values
        self.labels = df.label.values
        self.user2id = {u: i for i, u in enumerate(self.users)}

        self.filter_list = [
            w
            for w in self._top_words(df.text.values.tolist())
            if w not in stops and w in self.tok.vocab and w.isalpha()
        ][self.vocab_start : self.vocab_end]
        self.vocab_filter = torch.tensor(
            [t for t in self.tok.encode(self.filter_list) if t >= 2100]
        )

        df.update(df.text.astype(str).apply(lambda x: x.strip()).apply(self._truncate))
        self.texts = df.text.values

    def _truncate(self, text):
        if len(text) > 512:
            text = text[:256] + text[-256:]

        return text

    def _top_words(self, texts):
        vocab = Counter()
        for text in texts:
            vocab.update(text.strip().split())

        total = sum(vocab.values())
        vocab = {w: count / total for w, count in vocab.items()}

        w_counts = dict()
        for w in vocab:
            w_counts[w] = w_counts.get(w, 0) + vocab[w]

        w_top = sorted(w_counts.keys(), key=lambda x: w_counts[x], reverse=True)

        return w_top

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        user = self.users[idx]
        text = self.texts[idx]
        moment = self.deltas[idx]
        clazz = self.labels[idx]

        return user, text, moment, clazz


class TimeCollator:
    def __init__(self, user2id, tok, mlm_p=0.15):
        self.user2id = user2id
        self.tok = tok
        self.mlm_p = mlm_p

    def __call__(self, batch):
        u, t, m, c = zip(*batch)
        users = torch.tensor(np.array(list(map(lambda x: self.user2id[x], u))))
        moments = torch.tensor(np.array(m)).float()
        c = torch.tensor(np.array(c)).int()

        # Función que tokeniza y arregla los datos en el formato matricial numpy
        def f_app(texts):
            input_s = self.tok(
                texts.tolist(),
                return_tensors="pt",
                add_special_tokens=True,
                truncation=True,
                padding="max_length",
            )

            return np.array(
                [
                    input_s["input_ids"].numpy(),
                    input_s["token_type_ids"].numpy(),
                    input_s["attention_mask"].numpy(),
                ]
            )

        # target_axis = len(np.array(t).shape) - 1 #Dimensión de trabajo
        # Aplica la función auxiliar en la dimensión de trabajo
        ans = np.apply_along_axis(f_app, 0, t)
        # Desplegamos los datos de la dimensión de trabajo
        texts_pad, segs_pad, masks_pad = np.split(ans, ans.shape[0], axis=0)
        # Recuperamos el shape (batch, win, emb)
        texts_pad = torch.tensor(np.squeeze(texts_pad, axis=0))
        segs_pad = torch.tensor(np.squeeze(segs_pad, axis=0))
        masks_pad = torch.tensor(np.squeeze(masks_pad, axis=0))

        # Ahora tenemos que preparar los datos para MLM
        labels = texts_pad.clone()  # Clona los input_ids
        #
        p_matrix = torch.full(labels.shape, self.mlm_p)  # Matriz de probabilidad
        special_mask = np.apply_along_axis(
            lambda x: self.tok.get_special_tokens_mask(
                x, already_has_special_tokens=True
            ),
            -1,
            labels,
        )
        p_matrix.masked_fill_(
            torch.tensor(special_mask, dtype=torch.bool), value=0.0
        )  # Hacemos que se ignoren los tokens especiales
        padding_mask = labels.eq(self.tok.pad_token_id)
        p_matrix.masked_fill_(
            padding_mask, value=0.0
        )  # Hacemos que se ignore el token de padding
        masked_indices = torch.bernoulli(
            p_matrix
        ).bool()  # Usamos una distribución de bernoulli
        labels[
            ~masked_indices
        ] = -100  # Esto hace que no se tengan en cuenta los tokens no seleccionados para calcular la pérdida
        # Entiendo que hasta aquí se han ignorado una parte de los tokens especiales y de padding siguiendo una distribución de probabilidad

        indices_replaced = (
            torch.bernoulli(torch.full(labels.shape, 0.8)).bool() & masked_indices
        )  # Aplica una lógica de reemplazo de tokens
        texts_pad[indices_replaced] = self.tok.convert_tokens_to_ids(
            self.tok.mask_token
        )

        indices_random = (
            torch.bernoulli(torch.full(labels.shape, 0.5)).bool()
            & masked_indices
            & ~indices_replaced
        )  # Inserta tokens aleatorios??!?!?!
        random_words = torch.randint(len(self.tok), labels.shape, dtype=torch.long)
        texts_pad[indices_random] = random_words[indices_random]

        return labels, users, moments, c, texts_pad, masks_pad, segs_pad

In [ ]:
from typing import Any, Dict
from pydantic import BaseModel
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader
from transformers.models.bert import BertTokenizer
from sklearn.model_selection import train_test_split
from scipy.sparse import vstack

from rich import print as rprint

import time


class ModelConfig(BaseModel):
    # clazz: str=Field(...)
    lambda_a: float
    lambda_w: float
    hidden_size: int
    # timestamp_kind: str
    n_heads: int
    drop_prob: float
    n_layers: int
    time2vec_activation: str


class DCWEFilter(BaseFilter):
    def __init__(
        self,
        model: str,
        lr: float,
        n_epoch: int,
        batch_size: int,
        params: ModelConfig,
        deltas_f=["cosine"],
        vocab_start: int = 0,
        vocab_end: int = 10000,
    ):
        self.device = torch.device(
            "cuda:{}".format(0) if torch.cuda.is_available() else "cpu"
        )
        self.n_epoch = n_epoch
        self.batch_size = batch_size
        self.vocab_start = vocab_start
        self.vocab_end = vocab_end
        if not any([0 if x in DISTANCES else 1 for x in deltas_f]):
            self.deltas_f = deltas_f
        else:
            raise Exception(
                "Wrong deltas distance function. Available options are: {}.".format(
                    list(DISTANCES.keys())
                )
            )
        self.dcwe = DCWE(**params)  # type: ignore
        self.optimizer = AdamW(self.dcwe.parameters(), lr=lr)
        self.loss_fn = CrossEntropyLoss()
        self.all_models: Dict[float, Any] = dict()
        self.vocab_hash_map = None
        self.tk = BertTokenizer.from_pretrained("bert-base-uncased")

    def fit(self, x: XYData, y: XYData | None) -> float | None:
        print(f"Vocab {self.vocab_start}:{self.vocab_end}")
        data: pd.DataFrame = x.value
        aux_dataset = TimeDatasetNoWindow(
            data, tk=self.tk, vocab_start=self.vocab_start, vocab_end=self.vocab_end
        )
        print(f"Vocab filter len {len(aux_dataset.vocab_filter)}")
        self.vocab_hash_map = dict(
            zip(
                aux_dataset.vocab_filter.cpu().tolist(),
                range(len(aux_dataset.vocab_filter)),
            )
        )
        self.dcwe.vocab_filter = aux_dataset.vocab_filter.to(self.device)
        train_global_count = 0
        eval_global_count = 0
        train_dataset, valid_dataset = train_test_split(
            aux_dataset, train_size=7 / 8, test_size=1 / 8, shuffle=True
        )
        for epoch in range(1, self.n_epoch + 1):
            self.dcwe.train()
            losses = list()
            start_time = time.time()
            collator = TimeCollator(aux_dataset.user2id, aux_dataset.tok)
            train_loader = DataLoader(
                train_dataset,
                batch_size=self.batch_size,
                collate_fn=collator,
                shuffle=True,
            )
            valid_loader = DataLoader(
                valid_dataset,
                batch_size=self.batch_size,
                collate_fn=collator,
                shuffle=True,
            )
            for batch in tqdm(
                train_loader, desc=f"Runing train batches (epoch {epoch})"
            ):
                labels, _, moments, _, texts, masks, segs = batch

                labels = labels.to(self.device)
                texts = texts.to(self.device)
                masks = masks.to(self.device)
                segs = segs.to(self.device)
                moments = moments.to(self.device)

                self.dcwe.zero_grad()

                out = self.dcwe(texts, moments, masks, segs)
                loss = self.dcwe.loss(out, labels, self.loss_fn)

                losses.append(loss.item())
                loss.backward()

                self.optimizer.step()
                train_global_count += 1

            perplexity_train = np.exp(np.nanmean(losses))

            rprint(
                {
                    "perplexity_train": perplexity_train,
                    "epoch": epoch,
                    "train_time": (time.time() - start_time) / 60,
                }
            )

            self.dcwe.eval()
            losses = list()
            start_time = time.time()
            with torch.no_grad():
                for batch in tqdm(
                    valid_loader, desc=f"Evaluating model (epoch {epoch})"
                ):
                    labels, _, moments, _, texts, masks, segs = batch

                    labels = labels.to(self.device)
                    texts = texts.to(self.device)
                    masks = masks.to(self.device)
                    segs = segs.to(self.device)
                    moments = moments.to(self.device)

                    out = self.dcwe(texts, moments, masks, segs)
                    loss = self.dcwe.loss(out, labels, self.loss_fn)
                    losses.append(loss.item())
                    eval_global_count += 1

            perplexity_eval = np.exp(np.nanmean(losses))
            rprint(
                {
                    "perplexity_eval": perplexity_eval,
                    "epoch": epoch,
                    "eval_time": (time.time() - start_time) / 60,
                }
            )
            self.all_models[perplexity_eval] = self.dcwe.state_dict()

        valid_score, state_dict = max(self.all_models.items(), key=lambda xx: xx[0])
        print(f"Selected model with eval perplexity of {valid_score}")
        self.dcwe.load_state_dict(state_dict)

    def predict(self, x: XYData) -> XYData:
        all_deltas = {}
        data: pd.DataFrame = x.value
        x_dataset = TimeDatasetNoWindow(
            data, tk=self.tk, vocab_start=self.vocab_start, vocab_end=self.vocab_end
        )

        with torch.no_grad():
            collator = TimeCollator(x_dataset.user2id, x_dataset.tok)

            train_loader = DataLoader(
                x_dataset, batch_size=self.batch_size, collate_fn=collator
            )

            for i, batch in tqdm(list(enumerate(train_loader)), desc="Predicting..."):
                labels, users, moments, clazz, texts, masks, segs = batch

                labels = labels.to(self.device)
                texts = texts.to(self.device)
                masks = masks.to(self.device)
                segs = segs.to(self.device)
                moments = moments.to(self.device)

                _, _, output = self.dcwe(texts, moments, masks, segs)
                input_embs = self.dcwe.bert_emb_layer(texts)

                for metric in self.deltas_f:
                    aux = all_deltas.get(metric, [])
                    batch_matrix = self.dcwe.generate_deltas(
                        texts,
                        input_embs,
                        output.hidden_states[-1],
                        self.vocab_hash_map,
                        DISTANCES[metric],
                    )
                    aux.append(batch_matrix)
                    all_deltas[metric] = aux

            for metric in self.deltas_f:
                all_deltas[metric] = vstack(all_deltas[metric])

        return XYData.mock(all_deltas)

## The classifiers

This work is an early prediction task of the eRiks, this means we need to use some classifiers. Lets create a set of classifiers and wrap them in a the framework clases.

#### SVM

In [ ]:
from sklearn.svm import SVC


@Container.bind()
class ClassifierSVM(BaseFilter):
    def __init__(
        self,
        C,
        kernel,
        gamma,
        coef0,
        tol,
        decision_function_shape,
        class_weight_1: dict,
        probability,
    ):
        self.proba = probability
        self._model = SVC(
            C=C,
            kernel=kernel,
            gamma=gamma,
            coef0=coef0,
            tol=tol,
            decision_function_shape=decision_function_shape,
            class_weight={1: class_weight_1},
            probability=probability,
            random_state=43,
        )

    def fit(self, x: XYData, y: XYData | None):
        if y is None:
            raise ValueError("y must be provided for training")
        self._model.fit(x.value, y.value)

    def predict(self, x: XYData) -> XYData:
        if self.proba:
            result = list(map(lambda i: i[1], self._model.predict_proba(x.value)))
            return XYData.mock(result)
        else:
            result = self._model.predict(x.value)
            return XYData.mock(result)

#### Random Forest

In [ ]:
from typing import Literal
from sklearn.ensemble import RandomForestClassifier


class GaussianNaiveBayes(BaseFilter):
    def __init__(
        self,
        n_estimators=100,
        criterion: Literal["gini", "entropy", "log_loss"] = "gini",
        max_depth=2,
        min_samples_split=2,
        min_samples_leaf=1,
        max_features: float | Literal["sqrt", "log2"] = "sqrt",
        class_weight=None,
        proba=False,
    ):
        self.proba = proba
        self._model = RandomForestClassifier(
            n_estimators=n_estimators,
            criterion=criterion,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            max_features=max_features,
            class_weight=class_weight,
            random_state=0,
        )

    def fit(self, x: XYData, y: XYData | None):
        if y is None:
            raise ValueError("y must be provided for training")
        self._model.fit(x.value, y.value)

    def predict(self, x: XYData) -> XYData:
        if self.proba:
            result = list(map(lambda i: i[1], self._model.predict_proba(x.value)))
        else:
            result = self._model.predict(x.value)
        return XYData.mock(result)

#### XGBoost

In [ ]:
!pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 MB 102.3 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
from xgboost import XGBClassifier


class XGBClassifierPlugin(BaseFilter):
    def __init__(
        self,
        n_estimators,
        max_depth,
        learning_rate,
        min_child_weight,
        gamma,
        subsample,
        colsample_bytree,
        reg_alpha,
        reg_lambda,
        objective="binary:logistic",
    ):
        self.bst = XGBClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            learning_rate=learning_rate,
            min_child_weight=min_child_weight,
            gamma=gamma,
            subsample=subsample,
            colsample_bytree=colsample_bytree,
            reg_alpha=reg_alpha,
            reg_lambda=reg_lambda,  # L2 regularization term on weights, defaults to 1.0
            objective=objective,
        )

    def fit(self, x: XYData, y: XYData | None):
        if y is None:
            raise ValueError("y must be provided for training")
        self.bst.fit(x.value, y.value)

    def predict(self, x: XYData) -> XYData:
        return self.bst.predict(x.value)

## The metrics

Also because the early prediction task is a classification task, the metrics we will use will be: F1, Precision and Recall. But because of the early nature of the task we need to use some metrics that penalize the late decisións.

In [ ]:
from framework3.plugins.metrics import F1, Precission, Recall

f1 = F1()
precision = Precission()
recall = Recall()

ModuleNotFoundError: No module named 'tests.unit'

In [ ]:
from typing import Iterable
from sklearn.metrics import confusion_matrix
from framework3 import BaseMetric

from numpy import exp


@Container.bind()
class ERDE(BaseMetric):
    def __init__(self, count: Iterable, k: int = 5):
        self.k = k
        self.count = count

    def evaluate(
        self, x_data: XYData, y_true: XYData | None, y_pred: XYData
    ) -> float | np.ndarray:
        if y_true is None:
            raise ValueError("y_true must be provided for evaluation")

        all_erde = []
        _, _, _, tp = confusion_matrix(y_true.value, y_pred.value).ravel()
        for expected, result, count in list(
            zip(y_true.value, y_pred.value, self.count)
        ):
            if result == 1 and expected == 0:
                all_erde.append(float(tp) / len(y_true.value))
            elif result == 0 and expected == 1:
                all_erde.append(1.0)
            elif result == 1 and expected == 1:
                all_erde.append(1.0 - (1.0 / (1.0 + exp(count - self.k))))
            elif result == 0 and expected == 0:
                all_erde.append(0.0)
        return float(np.mean(all_erde) * 100)

## The pipeline

No it comes the most exiting part, the moment to integrate the filters in the pipeline. This step can be don incrementally wich is most convenient when developping a model but as far as we already know what we are doing we will join all the parts together in one step.